In [39]:
import numpy as np
import sys
sys.path.append("..")
from utils.data_utils import *
from ultralytics import YOLO
from tqdm import tqdm
import imutils
from collections import Counter, defaultdict

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
def generate_predictions(imgdir, model, dest_dir, conf_threshold = None, iou_nms_thresh=None, save_results = False):
    results_dict = defaultdict(lambda: defaultdict(list))
    c = 0
    totals = []
    for imgname in tqdm(os.listdir(imgdir)):
        if imgname != "LLW9QtUWRvOnKAS0I9HvjA_file.jpeg":continue
        imgpath = f"{imgdir}/{imgname}"
        
        img = np.array(Image.open(imgpath))
        img = imutils.resize(img, width = 1920)
        
        imgh, imgw, _ = img.shape
        print("h, w: ", imgh, imgw)
        results = model(img, imgsz=480, iou=iou_nms_thresh, verbose = False)

        result = results[0]
        
        
        #If no detections found
        if len(result.boxes.cpu().numpy()) == 0:
            results_dict[imgname]['bbox'] = []
            results_dict[imgname]['scores'] = []
            continue
        
        #Loop through the results
        #for result in results:
        
        
        boxes = result.boxes.cpu().numpy()
        print("YOLO bboxes: ", boxes.xyxy)
        print("YOLO confs: ", boxes.conf)
        
        for i, box in enumerate(boxes):
            conf = box.conf[0]
            
            if conf < conf_threshold:# If conf < threshold
                results_dict[imgname]['bbox'] = []
                results_dict[imgname]['scores'] = []
                continue
                
            bbox = box.xyxy[0].astype(int)
            results_dict[imgname]['bbox'].append(bbox.tolist())
            results_dict[imgname]['scores'].append(float(f"{conf:.4f}"))
            print(results_dict)
            
        break    
            
    print("Total Images predicted on: ", len(results_dict))
    if save_results:
        dump_json(results_dict, f"{dest_dir}/prediction_results.json", indent = 1)
    return results_dict

    

In [43]:
iteration = "v3_m"
save_results = False
conf_threshold = 0.1
iou_nms_thresh = 0.7

dest_dir = f"my_runs/lpblur/{iteration}/val/val_analysis_{str(iou_nms_thresh)}_{str(conf_threshold)}"
os.makedirs(dest_dir, exist_ok=True)

names = {0: 'licenseplate'}
#Load Model
model = YOLO(f"my_runs/lpblur/{iteration}/weights/best.pt")

#imgdir = "../datasets/LP_yolo_dataset/val/images"
imgdir = "../datasets/audit_data"

results_dict = generate_predictions(imgdir, model,dest_dir, conf_threshold = conf_threshold, iou_nms_thresh=iou_nms_thresh, save_results = save_results)


  0%|          | 0/2204 [00:00<?, ?it/s]

h, w:  1440 1920


 83%|████████▎ | 1826/2204 [00:00<00:00, 3858.87it/s]

YOLO bboxes:  [[     685.83      735.04      1204.1      871.01]]
YOLO confs:  [    0.83496]
defaultdict(<function generate_predictions.<locals>.<lambda> at 0x10523ca60>, {'LLW9QtUWRvOnKAS0I9HvjA_file.jpeg': defaultdict(<class 'list'>, {'bbox': [[685, 735, 1204, 871]], 'scores': [0.835]})})
Total Images predicted on:  1


In [31]:
c = np.array([0.80423])

idx = np.where(c>0.80)[0]

len(idx)

1

ModuleNotFoundError: No module named 'imutils'